In [1]:
%%capture
!apt-get update
!apt-get install libgdal-dev -y
!apt-get install python-gdal -y
!apt-get install python-numpy python-scipy -y
!pip install rasterio
!pip install fiona
!pip install geopandas
!pip install imgaug
!pip install -i https://test.pypi.org/simple/ gis-utils-pkg-dillhicks==0.0.4

#Importing GIS Software 
from rasterio import windows 
from rasterio.features import shapes 
from  gis_utils import raster 
import geopandas as gpd 
import rasterio 
import gdal 
import os 

In [2]:
import itertools
import os
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tqdm.autonotebook import tqdm
from imgaug import augmenters as iia

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import sys


In [3]:
%load_ext tensorboard

In [4]:
#loading training data

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
%%capture
!cp "/content/drive/Shared drives/SIO and E4E Mangroves /Data/Machine Learning/Training Images/Hybrid CNN/half-dataset.zip" /content/half-dataset.zip
!unzip "/content/half-dataset.zip" -d dataset

In [6]:
from skimage.transform import resize
from os.path import basename, dirname
from rasterio.plot import reshape_as_image
import glob
import random
from sklearn.model_selection import train_test_split

def load_images(image_dir, n_bands, sample=1, p_aug=0.25):

        
    num_images = (len(glob.glob1("/content/dataset/dataset/1","*.tif")) + len(glob.glob1("/content/dataset/dataset/0","*.tif")))/2

    index = 0
    labels = []
    images = [] 

    print("Finding All Files")
    #getting all filennames and labels for each file
    for root, dirs, files in os.walk(image_dir):
        for file in tqdm(files):
            if (file.endswith(".tif") and file.startswith("hr_")):
                file_path = os.path.join(root, file)
                label = int(basename(dirname(file_path)))
                labels.append(label)
                images.append(file_path)


    #shuffling the data
    c = list(zip(images, labels))

    random.shuffle(c)

    images, labels = zip(*c)

    #getting proportion of the data specified by sample
    images = images[0:int(num_images*sample)]
    labels = labels[0:int(num_images*sample)]

    labels = np.asarray(labels)

    #splitting data based on indices of data 
    x_ids = list(range(int(num_images*sample)))
    x_train_ids, x_test_ids, y_train, y_test = train_test_split(x_ids, labels, test_size = 0.2, random_state=42)

    #getting indices of training data to augment 
    n_train = len(x_train_ids)
    n_test = len(x_test_ids)

    if p_aug != 0:
        n_aug = int(len(x_train_ids)*p_aug)

        

    #creating memmaps
    x_hr_train = np.memmap('hr_train.memmap', dtype='float32', mode='w+', shape=(n_train + n_aug, 112,112,3))
    x_hr_test = np.memmap('hr_test.memmap', dtype='float32', mode='w+', shape=(n_test,112,112,3))

    x_lr_train = np.memmap('lr_train.memmap', dtype='float32', mode='w+', shape=(n_train + n_aug, n_bands))
    x_lr_test = np.memmap('lr_test.memmap', dtype='float32', mode='w+', shape=(n_test, n_bands))

    print("Loading Training Data")
    #fill memmap with unaugmented data
    for i, index in enumerate(tqdm(x_train_ids)):
        
        file_path = images[index]
    
        hr_image, _ = raster.load_image(file_path)
        lr_image, _ = raster.load_image(file_path.replace("hr_", "lr_"))
        
        x_hr_train[i,:,:,:] = resize(reshape_as_image(hr_image.read()), (112,112,3))
        x_lr_train[i,:] = np.squeeze(np.squeeze(reshape_as_image(lr_image.read())))
    
    
    #create augmentor
    seq = iia.Sequential([
                        iia.Multiply((0.75, 1.1))
                        ])

    #load augmented data into memmap
    if p_aug != 0:
        print("Generating Augmented Data")
        for i in tqdm(range(n_aug)):
            
            cur_image = np.expand_dims(x_hr_train[i,:,:,:], axis=0)

            x_hr_train[i + n_train,:,:,:] = np.squeeze(seq(images=cur_image))
            x_lr_train[i + n_train,:] = x_lr_train[i,:]

    #adding the labels for the augmented data 
    y_train = np.concatenate((y_train, y_train[0:n_aug]))

    print("Loading Test Data")
    #generating the test data
    for i, index in enumerate(tqdm(x_test_ids)):

        file_path = images[index]

        hr_image, _ = raster.load_image(file_path)
        lr_image, _ = raster.load_image(file_path.replace("hr_", "lr_"))

        x_hr_test[i,:,:,:] = resize(reshape_as_image(hr_image.read()), (112,112,3))
        x_lr_test[i,:] = np.squeeze(np.squeeze(reshape_as_image(lr_image.read())))


    return x_hr_train, x_hr_test, x_lr_train, x_lr_test, y_train, y_test

In [7]:
from tensorflow.keras.layers import InputLayer, Dense, concatenate

def create_cnn(module_handle, module_name, image_size, n_2):
    model = tf.keras.Sequential([
    InputLayer(input_shape = IMAGE_SIZE + (3,)),
    hub.KerasLayer(MODULE_HANDLE, trainable = True, name = module_name),
    Dense(50)])
    return model

In [8]:
def create_mlp(n_bands, n_1):
    model = tf.keras.Sequential([
       tf.keras.layers.Dense(32, input_dim=n_bands)                         
    ])
    return model

In [9]:
def show_model(model):
    tf.keras.utils.plot_model(model)
    model.summary()

In [10]:
def create_hybrid(MODULE_HANDLE, IMAGE_SIZE, BATCH_SIZE, n_1, n_2, n_bands):
    feature_extractor = create_cnn(MODULE_HANDLE,"Resnet50V2", IMAGE_SIZE,n_2)
    mlp = create_mlp(n_bands,n_1)

    combined = concatenate([mlp.output, feature_extractor.output])
    x = Dense(2, activation="sigmoid")(combined)


    model = tf.keras.Sequential()
    model = tf.keras.models.Model(inputs=[mlp.input, feature_extractor.input], outputs=x)
   
    model.compile(
    optimizer=tf.keras.optimizers.RMSprop(), 
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True, label_smoothing=0.1),
    metrics=['accuracy'])

    return model

In [11]:
def create_base(MODULE_HANDLE, IMAGE_SIZE, BATCH_SIZE):
    model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
    hub.KerasLayer(MODULE_HANDLE, trainable=False),
    tf.keras.layers.Dense(2, activation="sigmoid")
    ])
    model.build((None,)+IMAGE_SIZE+(3,))
    return model


In [12]:
from os.path import join

for root, dirs, files in os.walk("/content/dataset/dataset/0"):
    firstfile = sorted(files)[-1]
    test_file =  os.path.join(root, firstfile)

test_img, _ = raster.load_image(test_file)
n_bands = test_img.count

In [ ]:
x_hr_train, x_hr_test, x_lr_train, x_lr_test, y_train, y_test = load_images("dataset",n_bands, sample=0.5)

print(y_train.shape)
print(x_hr_train.shape)
print(x_lr_train.shape)

Finding All Files



Loading Training Data


In [ ]:
print(y_train.shape)
print(y_test.shape)

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
MODULE_HANDLE = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"
IMAGE_SIZE = (112,112)
BATCH_SIZE = 64

n_1 = 32
n_2 = 256
 
model = create_hybrid(MODULE_HANDLE, IMAGE_SIZE, BATCH_SIZE, n_1, n_2, n_bands)
history = model.fit(x = [x_lr_train, x_hr_train], y = y_train, validation_data = ([x_lr_test, x_hr_test], y_test), epochs = 30)
model.save("/content/drive/Shared drives/SIO and E4E Mangroves /Data/Machine Learning/Models/Hybrid CNN/efficientnetb0-ndwi+ndvi")

In [ ]:
from sklearn.metrics import confusion_matrix
def pixel_iou(y_true,y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    iou = tp / (tp + fn + fp)
    return iou

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("/content/drive/Shared drives/SIO and E4E Mangroves /Data/Machine Learning/Models/Hybrid CNN/efficientnetb0-ndwi+ndvi")

In [ ]:
y_predict = np.argmax(model.predict([x_lr_test,x_hr_test]), axis=1)

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(y_test,y_predict))
print("IOU Score: ", pixel_iou(y_test,y_predict))